# Real-time Implementation and Automation with FXCM 

## Recap: Historical Data, real-time Data and Orders

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
api.get_candles("EUR/USD", start = "2020-07-01", end = "2020-07-31",  period = "D1")

In [ ]:
api.get_candles("EUR/USD", number = 10, period = "m1")

In [ ]:
api.subscribe_market_data("EUR/USD")

In [ ]:
api.get_subscribed_symbols()

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))

In [ ]:
api.subscribe_market_data("EUR/USD", (print_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
api.get_open_positions()

In [ ]:
api.create_market_buy_order("EUR/USD", 100)

In [ ]:
order = api.create_market_buy_order("EUR/USD", 100)

In [ ]:
order.get_time()

In [ ]:
order.get_isBuy()

In [ ]:
order.get_currency()

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
api.get_open_positions()[col]

In [ ]:
api.create_market_sell_order("EUR/USD", 200)

In [ ]:
api.get_open_positions()[col]

In [ ]:
api.close_all_for_symbol("EUR/USD")

In [ ]:
api.get_open_positions()

In [ ]:
api.get_closed_positions()[col]

In [ ]:
api.get_closed_positions_summary()[col]

In [ ]:
api.get_accounts().T

In [ ]:
api.close()

## Collecting and storing real-time tick data

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
def print_data(data, dataframe):
    print(dataframe)

In [ ]:
api.subscribe_market_data("EUR/USD", (print_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
tick_data = None

In [ ]:
def get_tick_data(data, dataframe):
    
    global tick_data # global variable tick_data
    
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))
    
    tick_data = dataframe.iloc[:, :2]

In [ ]:
api.subscribe_market_data("EUR/USD", (get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
tick_data

In [ ]:
tick_data.info()

In [ ]:
api.close()

## Storing and resampling real-time tick data

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

In [ ]:
tick_data = None
resamp = None
bar_length = "10s"

In [ ]:
def get_tick_data(data, dataframe):
    
    global tick_data, resamp # global variables
    
    print('%3d | %s | %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1]))
    
    tick_data = dataframe.iloc[:, :2]
    resamp = tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1]
    resamp["Mid"] = (resamp.Ask + resamp.Bid)/2

In [ ]:
api.subscribe_market_data("EUR/USD", (get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
tick_data

In [ ]:
resamp

In [ ]:
api.close()

## Implementing a Contrarian Strategy in Real-Time (Part 1)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
tick_data = None
resamp = None
bar_length = "5s"
window = 1
ticks = 0

In [ ]:
def con_trader(data, dataframe):
    
    global tick_data, resamp, ticks # global variables
    
    ticks += 1
    print(ticks, end = " ")
    
    # store and resample tick data
    tick_data = dataframe.iloc[:, :2]
    resamp = tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1]
    resamp["Mid"] = (resamp.Ask + resamp.Bid)/2
    
    # prepare data & define strategy
    resamp["returns"] = np.log(resamp.Mid / resamp.Mid.shift(1))
    resamp["position"] = -np.sign(resamp.returns.rolling(window).mean())

In [ ]:
for i in range(10):
    print(i)

In [ ]:
api.subscribe_market_data("EUR/USD", (con_trader, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
tick_data

In [ ]:
resamp

In [ ]:
api.close()

## Implementing a Contrarian Strategy in Real-Time (Part 2)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
tick_data = None
resamp = None
bar_length = "5s"
window = 1
ticks = 0
position = 0
units = 100
instrument = "EUR/USD"
min_length = window + 1

The following code assumes a __Netting Account__ (NOT Hedging)

In [ ]:
def con_trader(data, dataframe):
    
    global tick_data, resamp, ticks, position, min_length # global variables
    
    ticks += 1
    print(ticks, end = " ")
    
    # store and resample tick data
    tick_data = dataframe.iloc[:, :2]
    resamp = tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1]
    resamp["Mid"] = (resamp.Ask + resamp.Bid)/2
    
    # prepare data & define strategy
    resamp["returns"] = np.log(resamp.Mid / resamp.Mid.shift(1))
    resamp["position"] = -np.sign(resamp.returns.rolling(window).mean())
 
    # executing trades
    if len(resamp) > min_length - 1: # if a new bar is added: go through if/elif
        min_length += 1
        if resamp["position"].iloc[-1] == 1: # if signal is long
            if position == 0:
                order = api.create_market_buy_order(instrument, units) # buy 1 * units if position is neutral ("1 trade")
                print(2*"\n" + "{} | GO LONG".format(str(order.get_time())) + "\n")
            elif position == -1:
                order = api.create_market_buy_order(instrument, 2 * units) # buy 2 * units if position is short ("2 trades") 
                print(2*"\n" + "{} | GO LONG".format(str(order.get_time())) + "\n")
            position = 1
        elif resamp["position"].iloc[-1] == -1: # if signal is short
            if position == 0:
                order = api.create_market_sell_order(instrument, units) # sell 1 * units if position is neutral ("1 trade")
                print(2*"\n" + "{} | GO SHORT".format(str(order.get_time())) + "\n")
            elif position == 1:
                order = api.create_market_sell_order(instrument, 2 * units) # sell 2 * units if position is long ("2 trades")
                print(2*"\n" + "{} | GO SHORT".format(str(order.get_time())) + "\n")
            position = -1        

## Implementing a Contrarian Strategy in Real-Time (Part 3)

In [ ]:
api.subscribe_market_data("EUR/USD", (con_trader, ))

In [ ]:
api.close_all_for_symbol(instrument)
api.unsubscribe_market_data("EUR/USD")

In [ ]:
api.get_open_positions()

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
api.get_closed_positions()[col][-3:]

In [ ]:
api.get_closed_positions_summary()[col]

In [ ]:
resamp

In [ ]:
api.close()

## Implementing a Contrarian Strategy in Real-Time (Part 4)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
tick_data = None
resamp = None
bar_length = "5s"
window = 1
ticks = 0
position = 0
units = 100
instrument = "EUR/USD"
min_length = window + 1

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
def con_trader(data, dataframe):
    
    global tick_data, resamp, ticks, position, min_length # global variables
    
    ticks += 1
    print(ticks, end = " ")
    
    # store and resample tick data
    tick_data = dataframe.iloc[:, :2]
    resamp = tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1]
    resamp["Mid"] = (resamp.Ask + resamp.Bid)/2
    
    # prepare data & define strategy
    resamp["returns"] = np.log(resamp.Mid / resamp.Mid.shift(1))
    resamp["position"] = -np.sign(resamp.returns.rolling(window).mean())
 
    # executing trades
    if len(resamp) > min_length - 1: # if a new bar is added: go through if/elif
        min_length += 1
        if resamp["position"].iloc[-1] == 1: # if signal is long
            if position == 0:
                order = api.create_market_buy_order(instrument, units) # buy 1 * units if position is neutral ("1 trade")
                print(2*"\n" + "{} | GO LONG | unreal P&L: {}".format(str(order.get_time()), api.get_open_positions().grossPL.sum()) + "\n")
            elif position == -1:
                order = api.create_market_buy_order(instrument, 2 * units) # buy 2 * units if position is short ("2 trades") 
                print(2*"\n" + "{} | GO LONG | unreal P&L: {}".format(str(order.get_time()), api.get_open_positions().grossPL.sum()) + "\n")
            position = 1
        elif resamp["position"].iloc[-1] == -1: # if signal is short
            if position == 0:
                order = api.create_market_sell_order(instrument, units) # sell 1 * units if position is neutral ("1 trade")
                print(2*"\n" + "{} | GO SHORT | unreal P&L: {}".format(str(order.get_time()), api.get_open_positions().grossPL.sum()) + "\n")
            elif position == 1:
                order = api.create_market_sell_order(instrument, 2 * units) # sell 2 * units if position is long ("2 trades")
                print(2*"\n" + "{} | GO SHORT | unreal P&L: {}".format(str(order.get_time()), api.get_open_positions().grossPL.sum()) + "\n")
            position = -1    
            
    # define trading stop
    if ticks > 50:
        api.close_all_for_symbol(instrument)
        api.unsubscribe_market_data("EUR/USD")
        print(2*"\n" + "{} | GO NEUTRAL".format(str(tick_data.index[-1])) + "\n")
        print(api.get_closed_positions_summary()[col])

In [ ]:
api.subscribe_market_data("EUR/USD", (con_trader, ))

In [ ]:
api.close()

## Improving Efficiency

In [ ]:
import pandas as pd
import fxcmpy

In [ ]:
api = fxcmpy.fxcmpy(config_file = "fxcm.cfg")

In [ ]:
tick_data = None
resamp = None
bar_length = "5s"
ticks = 0

In [ ]:
def get_tick_data(data, dataframe):
    
    global tick_data, resamp, ticks # global variables
    
    ticks += 1
    
    if ticks % 5 == 0:
        print(ticks, end = " ")

        tick_data = dataframe.iloc[:, :2]
        resamp = tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1]
        resamp["Mid"] = (resamp.Ask + resamp.Bid)/2

In [ ]:
api.subscribe_market_data("EUR/USD", (get_tick_data, ))

In [ ]:
api.unsubscribe_market_data("EUR/USD")

In [ ]:
tick_data

In [ ]:
resamp

In [ ]:
api.close()

## Working with historical data and real-time tick data (Part 1)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
from datetime import datetime, timedelta

In [ ]:
instrument = "EUR/USD"

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
api.get_candles("EUR/USD", start = "2020-12-28", end = "2020-12-29",  period = "m1",
                columns = ["bidclose", "askclose"] )

In [ ]:
now = datetime.utcnow()
now

In [ ]:
yesterday = now - timedelta(days = 1)
yesterday

In [ ]:
df = api.get_candles("EUR/USD", start = yesterday, end = now,  period = "m1",
                     columns = ["bidclose", "askclose"])
df

In [ ]:
df[instrument] = (df.bidclose + df.askclose) / 2

In [ ]:
df = df[instrument].to_frame()

In [ ]:
df

In [ ]:
api.close()

## Working with historical data and real-time tick data (Part 2)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
from datetime import datetime, timedelta

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
hist_data = None
tick_data = None
raw_data = None
full_data = None
bar_length = "2min"
window = 20
ticks = 0
position = 0
units = 100
instrument = "EUR/USD"
min_length = None

In [ ]:
def get_most_recent(days = 5):
    
    global hist_data
    
    now = datetime.utcnow()
    past = now - timedelta(days = days)
    df = api.get_candles(instrument, start = past, end = now,  period = "m1",
                         columns = ["bidclose", "askclose"])
    df[instrument] = (df.bidclose + df.askclose) / 2
    df = df[instrument].to_frame()
    hist_data = df.resample(bar_length, label = "right").last().dropna().iloc[:-1]

In [ ]:
def con_trader(data, dataframe):
    
    global hist_data, tick_data, raw_data, full_data, ticks, position, min_length # global variables
    
    ticks += 1
    print(ticks, end = " ")
        
    
    # store and resample tick data and join with historical data
    tick_data = dataframe.iloc[:, :2].copy()
    tick_data[instrument] = (tick_data.Ask + tick_data.Bid)/2
    tick_data = tick_data[instrument].to_frame()
    raw_data = hist_data.append(tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1])
    
 
    # executing trades
    #...
    #...
    #...

In [ ]:
get_most_recent(days = 5)
api.subscribe_market_data(instrument, (con_trader, ))

In [ ]:
api.unsubscribe_market_data(instrument)

In [ ]:
hist_data

In [ ]:
tick_data

In [ ]:
raw_data

In [ ]:
api.close()

## Working with historical data and real-time tick data (Part 3)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
from datetime import datetime, timedelta

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
hist_data = None
tick_data = None
raw_data = None
full_data = None
bar_length = "2min"
window = 20
ticks = 0
position = 0
units = 100
instrument = "EUR/USD"
min_length = None

In [ ]:
def get_most_recent(days = 5):
    
    global hist_data, min_length
    
    now = datetime.utcnow()
    past = now - timedelta(days = days)
    df = api.get_candles(instrument, start = past, end = now,  period = "m1",
                         columns = ["bidclose", "askclose"])
    df[instrument] = (df.bidclose + df.askclose) / 2
    df = df[instrument].to_frame()
    hist_data = df.resample(bar_length, label = "right").last().dropna().iloc[:-1]
    min_length = len(hist_data) + 1

In [ ]:
def prepare_data():
    
    global full_data
    
    df = raw_data.copy()
    df["returns"] = np.log(df[instrument] / df[instrument].shift())
    df["position"] = -np.sign(df.returns.rolling(window).mean())
    full_data = df.copy()

In [ ]:
def con_trader(data, dataframe):
    
    global hist_data, tick_data, raw_data, full_data, ticks, position, min_length # global variables
    
    ticks += 1
    print(ticks, end = " ")
        
    
    # store and resample tick data and join with historical data
    tick_data = dataframe.iloc[:, :2].copy()
    tick_data[instrument] = (tick_data.Ask + tick_data.Bid)/2
    tick_data = tick_data[instrument].to_frame()
    raw_data = hist_data.append(tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1])
    
 
    # only if full new bar has been added:
    if len(raw_data) > min_length - 1:
        min_length += 1
        prepare_data()
            
        # order and trades...
        # if full_data["position"].iloc[-1] == 1:
            #if position == 0:
                
                
                

In [ ]:
get_most_recent(days = 5)
api.subscribe_market_data(instrument, (con_trader, ))

In [ ]:
api.unsubscribe_market_data(instrument)

In [ ]:
raw_data

In [ ]:
full_data

In [ ]:
api.close()

##########################################################################################################

(you can find the corresponding Video for the following lines of code in __Section 23__: Cloud Deployment (AWS) | Scheduling Trading Sessions | Full Automation)

## How to Time/Schedule the End of a Trading Session (Simple Contrarian)

In [ ]:
import pandas as pd
import numpy as np
import fxcmpy
import datetime as dt

In [ ]:
dt.datetime.utcnow().time() # current utc time

In [ ]:
dt.time(17, 45) # desired end of trading session (in utc time)

In [ ]:
if dt.datetime.utcnow().time() >= dt.time(17, 45):
    print("Stop the Trading Session!")

In [ ]:
api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [ ]:
tick_data = None
resamp = None
bar_length = "5s"
window = 1
ticks = 0
position = 0
units = 100
instrument = "EUR/USD"
min_length = window + 1

In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
def con_trader(data, dataframe):
    
    global tick_data, resamp, ticks, position, min_length # global variables
    
    ticks += 1
    print(ticks, end = " ")
    
    # store and resample tick data
    tick_data = dataframe.iloc[:, :2]
    resamp = tick_data.resample(bar_length, label = "right").last().ffill().iloc[:-1]
    resamp["Mid"] = (resamp.Ask + resamp.Bid)/2
    
    # prepare data & define strategy
    resamp["returns"] = np.log(resamp.Mid / resamp.Mid.shift(1))
    resamp["position"] = -np.sign(resamp.returns.rolling(window).mean())
 
    # executing trades
    if len(resamp) > min_length - 1: # if a new bar is added: go through if/elif
        min_length += 1
        if resamp["position"].iloc[-1] == 1: # if signal is long
            if position == 0:
                order = api.create_market_buy_order(instrument, units) # buy 1 * units if position is neutral ("1 trade")
                print(2*"\n" + "{} | GO LONG | unreal P&L: {}".format(str(order.get_time()), api.get_open_positions().grossPL.sum()) + "\n")
            elif position == -1:
                order = api.create_market_buy_order(instrument, 2 * units) # buy 2 * units if position is short ("2 trades") 
                print(2*"\n" + "{} | GO LONG | unreal P&L: {}".format(str(order.get_time()), api.get_open_positions().grossPL.sum()) + "\n")
            position = 1
        elif resamp["position"].iloc[-1] == -1: # if signal is short
            if position == 0:
                order = api.create_market_sell_order(instrument, units) # sell 1 * units if position is neutral ("1 trade")
                print(2*"\n" + "{} | GO SHORT | unreal P&L: {}".format(str(order.get_time()), api.get_open_positions().grossPL.sum()) + "\n")
            elif position == 1:
                order = api.create_market_sell_order(instrument, 2 * units) # sell 2 * units if position is long ("2 trades")
                print(2*"\n" + "{} | GO SHORT | unreal P&L: {}".format(str(order.get_time()), api.get_open_positions().grossPL.sum()) + "\n")
            position = -1    
            
    # define trading stop
    if dt.datetime.utcnow().time() >= dt.time(17, 45):
        api.close_all_for_symbol(instrument)
        api.unsubscribe_market_data("EUR/USD")
        print(2*"\n" + "{} | GO NEUTRAL".format(str(tick_data.index[-1])) + "\n")
        print(api.get_closed_positions_summary()[col])

In [ ]:
api.subscribe_market_data("EUR/USD", (con_trader, ))

In [ ]:
api.close()